In [7]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Flatten



In [8]:
train_dir = 'Project101/final_dataset/train'
val_dir = 'Project101/final_dataset/validation'
test_dir = 'Project101/final_dataset/test'

In [9]:


# Load and preprocess the Food 101 dataset
# Make sure to have the dataset in the correct structure (train/val subdirectories for each class)
# You might need to resize images to (299, 299) as InceptionV3 requires this size

# Use the built-in preprocessing function for InceptionV3

from tensorflow.keras.applications.densenet import preprocess_input
train_datagen = ImageDataGenerator(                            
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2
  )

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)



Found 75750 images belonging to 101 classes.
Found 10100 images belonging to 101 classes.
Found 15150 images belonging to 101 classes.


In [10]:
# Load the pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape = (224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add new classification layers
x = base_model.output
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(101, activation='softmax')(x)

# Define the new model
model = Model(inputs=base_model.input, outputs=outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

29084464/29084464 [==============================] - 6s 0us/step


In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

 n)                                                                                               
                                                                                                  
 conv2_block4_1_conv (Conv2D)   (None, 56, 56, 128)  20480       ['conv2_block4_0_relu[0][0]']    
                                                                                                  
 conv2_block4_1_bn (BatchNormal  (None, 56, 56, 128)  512        ['conv2_block4_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block4_1_relu (Activatio  (None, 56, 56, 128)  0          ['conv2_block4_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_blo

 conv3_block2_0_relu (Activatio  (None, 28, 28, 160)  0          ['conv3_block2_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_1_conv (Conv2D)   (None, 28, 28, 128)  20480       ['conv3_block2_0_relu[0][0]']    
                                                                                                  
 conv3_block2_1_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block2_1_bn[0][0]']      
 n)                                                                                               
          

                                                                                                  
 conv3_block6_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block6_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block6_2_conv (Conv2D)   (None, 28, 28, 32)   36864       ['conv3_block6_1_relu[0][0]']    
                                                                                                  
 conv3_block6_concat (Concatena  (None, 28, 28, 320)  0          ['conv3_block5_concat[0][0]',    
 te)                                                              'conv3_block6_2_conv[0][0]']    
                                                                                                  
 conv3_block7_0_bn (BatchNormal  (None, 28, 28, 320)  1280       ['conv3_block6_concat[0][0]']    
 ization) 

 ate)                                                             'conv3_block10_2_conv[0][0]']   
                                                                                                  
 conv3_block11_0_bn (BatchNorma  (None, 28, 28, 448)  1792       ['conv3_block10_concat[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv3_block11_0_relu (Activati  (None, 28, 28, 448)  0          ['conv3_block11_0_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv3_block11_1_conv (Conv2D)  (None, 28, 28, 128)  57344       ['conv3_block11_0_relu[0][0]']   
                                                                                                  
 conv3_blo

...

                                                                                                  
 conv4_block24_2_conv (Conv2D)  (None, 14, 14, 32)   36864       ['conv4_block24_1_relu[0][0]']   
                                                                                                  
 conv4_block24_concat (Concaten  (None, 14, 14, 1024  0          ['conv4_block23_concat[0][0]',   
 ate)                           )                                 'conv4_block24_2_conv[0][0]']   
                                                                                                  
 pool4_bn (BatchNormalization)  (None, 14, 14, 1024  4096        ['conv4_block24_concat[0][0]']   
                                )                                                                 
                                                                                                  
 pool4_relu (Activation)        (None, 14, 14, 1024  0           ['pool4_bn[0][0]']               
          

                                                                                                  
 conv5_block4_1_relu (Activatio  (None, 7, 7, 128)   0           ['conv5_block4_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block4_2_conv (Conv2D)   (None, 7, 7, 32)     36864       ['conv5_block4_1_relu[0][0]']    
                                                                                                  
 conv5_block4_concat (Concatena  (None, 7, 7, 640)   0           ['conv5_block3_concat[0][0]',    
 te)                                                              'conv5_block4_2_conv[0][0]']    
                                                                                                  
 conv5_block5_0_bn (BatchNormal  (None, 7, 7, 640)   2560        ['conv5_block4_concat[0][0]']    
 ization) 

 te)                                                              'conv5_block8_2_conv[0][0]']    
                                                                                                  
 conv5_block9_0_bn (BatchNormal  (None, 7, 7, 768)   3072        ['conv5_block8_concat[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block9_0_relu (Activatio  (None, 7, 7, 768)   0           ['conv5_block9_0_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block9_1_conv (Conv2D)   (None, 7, 7, 128)    98304       ['conv5_block9_0_relu[0][0]']    
                                                                                                  
 conv5_blo

                                                                                                  
 conv5_block13_1_conv (Conv2D)  (None, 7, 7, 128)    114688      ['conv5_block13_0_relu[0][0]']   
                                                                                                  
 conv5_block13_1_bn (BatchNorma  (None, 7, 7, 128)   512         ['conv5_block13_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv5_block13_1_relu (Activati  (None, 7, 7, 128)   0           ['conv5_block13_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv5_block13_2_conv (Conv2D)  (None, 7, 7, 32)     36864       ['conv5_block13_1_relu[0][0]']   
          

__________________________________________________________________________________________________


In [12]:
# Define callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint_callback = ModelCheckpoint('dense_transfer_learning_model.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, mode='min', verbose=1)


In [13]:
import time
start_time = time.time()

In [14]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model('dense_transfer_learning_model.h5')

In [15]:
history2 = model.fit(
        train_generator,
        epochs=300,
        validation_data=val_generator,
        callbacks=[checkpoint_callback, early_stopping_callback, reduce_lr]
)

Epoch 1/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7912 - accuracy: 0.2736
Epoch 1: val_loss improved from inf to 2.77462, saving model to dense_transfer_learning_model.h5
4735/4735 [==============================] - 823s 173ms/step - loss: 2.7912 - accuracy: 0.2736 - val_loss: 2.7746 - val_accuracy: 0.2759 - lr: 1.0000e-04
Epoch 2/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7884 - accuracy: 0.2735
Epoch 2: val_loss improved from 2.77462 to 2.72756, saving model to dense_transfer_learning_model.h5
4735/4735 [==============================] - 701s 148ms/step - loss: 2.7884 - accuracy: 0.2735 - val_loss: 2.7276 - val_accuracy: 0.2852 - lr: 1.0000e-04
Epoch 3/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7898 - accuracy: 0.2749
Epoch 3: val_loss did not improve from 2.72756
4735/4735 [==============================] - 703s 148ms/step - loss: 2.7898 - accuracy: 0.2749 - val_loss: 2.7308 - val_accuracy: 0.2833 - lr: 1.0000e

Epoch 27/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7733 - accuracy: 0.2765
Epoch 27: val_loss did not improve from 2.71124
4735/4735 [==============================] - 690s 146ms/step - loss: 2.7733 - accuracy: 0.2765 - val_loss: 2.7434 - val_accuracy: 0.2810 - lr: 1.0000e-04
Epoch 28/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7717 - accuracy: 0.2777
Epoch 28: val_loss did not improve from 2.71124
4735/4735 [==============================] - 696s 147ms/step - loss: 2.7717 - accuracy: 0.2777 - val_loss: 2.7278 - val_accuracy: 0.2828 - lr: 1.0000e-04
Epoch 29/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7744 - accuracy: 0.2770
Epoch 29: val_loss did not improve from 2.71124
4735/4735 [==============================] - 693s 146ms/step - loss: 2.7744 - accuracy: 0.2770 - val_loss: 2.7183 - val_accuracy: 0.2805 - lr: 1.0000e-04
Epoch 30/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7714 - accuracy: 0

In [ ]:
# Train the model
history = model.fit(
        train_generator,
        epochs=300,
        validation_data=val_generator,
        callbacks=[checkpoint_callback, early_stopping_callback, reduce_lr]
)


Epoch 1/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7320 - accuracy: 0.2876
Epoch 1: val_loss improved from 2.71124 to 2.69438, saving model to dense_transfer_learning_model.h5
4735/4735 [==============================] - 692s 146ms/step - loss: 2.7320 - accuracy: 0.2876 - val_loss: 2.6944 - val_accuracy: 0.2914 - lr: 1.0000e-05
Epoch 2/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7292 - accuracy: 0.2893
Epoch 2: val_loss improved from 2.69438 to 2.69434, saving model to dense_transfer_learning_model.h5
4735/4735 [==============================] - 692s 146ms/step - loss: 2.7292 - accuracy: 0.2893 - val_loss: 2.6943 - val_accuracy: 0.2924 - lr: 1.0000e-05
Epoch 3/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7271 - accuracy: 0.2888
Epoch 3: val_loss did not improve from 2.69434
4735/4735 [==============================] - 693s 146ms/step - loss: 2.7271 - accuracy: 0.2888 - val_loss: 2.6953 - val_accuracy: 0.2923 - lr: 1.0

4735/4735 [==============================] - ETA: 0s - loss: 2.7176 - accuracy: 0.2925
Epoch 26: val_loss did not improve from 2.67872
4735/4735 [==============================] - 697s 147ms/step - loss: 2.7176 - accuracy: 0.2925 - val_loss: 2.6809 - val_accuracy: 0.2935 - lr: 1.0000e-05
Epoch 27/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7209 - accuracy: 0.2920
Epoch 27: val_loss did not improve from 2.67872
4735/4735 [==============================] - 697s 147ms/step - loss: 2.7209 - accuracy: 0.2920 - val_loss: 2.6797 - val_accuracy: 0.2969 - lr: 1.0000e-05
Epoch 28/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7212 - accuracy: 0.2906
Epoch 28: val_loss did not improve from 2.67872
4735/4735 [==============================] - 695s 147ms/step - loss: 2.7212 - accuracy: 0.2906 - val_loss: 2.6847 - val_accuracy: 0.2924 - lr: 1.0000e-05
Epoch 29/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7135 - accuracy: 0.2916
Epoch 2

4735/4735 [==============================] - 703s 149ms/step - loss: 2.7133 - accuracy: 0.2918 - val_loss: 2.6751 - val_accuracy: 0.2959 - lr: 1.0000e-05
Epoch 53/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7128 - accuracy: 0.2916
Epoch 53: val_loss did not improve from 2.67515
4735/4735 [==============================] - 699s 148ms/step - loss: 2.7128 - accuracy: 0.2916 - val_loss: 2.6777 - val_accuracy: 0.2939 - lr: 1.0000e-05
Epoch 54/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7108 - accuracy: 0.2942
Epoch 54: val_loss did not improve from 2.67515
4735/4735 [==============================] - 695s 147ms/step - loss: 2.7108 - accuracy: 0.2942 - val_loss: 2.6768 - val_accuracy: 0.2966 - lr: 1.0000e-05
Epoch 55/300
4735/4735 [==============================] - ETA: 0s - loss: 2.7110 - accuracy: 0.2929
Epoch 55: val_loss did not improve from 2.67515
4735/4735 [==============================] - 693s 146ms/step - loss: 2.7110 - accuracy: 0.2929

Epoch 77/300
4735/4735 [==============================] - ETA: 0s - loss: 2.6262 - accuracy: 0.3201
Epoch 77: val_loss improved from 2.59229 to 2.59099, saving model to dense_transfer_learning_model.h5
4735/4735 [==============================] - 704s 149ms/step - loss: 2.6262 - accuracy: 0.3201 - val_loss: 2.5910 - val_accuracy: 0.3246 - lr: 1.0000e-05
Epoch 78/300
4735/4735 [==============================] - ETA: 0s - loss: 2.6282 - accuracy: 0.3199
Epoch 78: val_loss did not improve from 2.59099
4735/4735 [==============================] - 695s 147ms/step - loss: 2.6282 - accuracy: 0.3199 - val_loss: 2.5925 - val_accuracy: 0.3250 - lr: 1.0000e-05
Epoch 79/300
4735/4735 [==============================] - ETA: 0s - loss: 2.6254 - accuracy: 0.3201
Epoch 79: val_loss improved from 2.59099 to 2.58817, saving model to dense_transfer_learning_model.h5
4735/4735 [==============================] - 694s 147ms/step - loss: 2.6254 - accuracy: 0.3201 - val_loss: 2.5882 - val_accuracy: 0.3255 - l

4735/4735 [==============================] - ETA: 0s - loss: 2.5872 - accuracy: 0.3290
Epoch 102: val_loss did not improve from 2.55274
4735/4735 [==============================] - 693s 146ms/step - loss: 2.5872 - accuracy: 0.3290 - val_loss: 2.5569 - val_accuracy: 0.3311 - lr: 1.0000e-05
Epoch 103/300
4735/4735 [==============================] - ETA: 0s - loss: 2.5863 - accuracy: 0.3282
Epoch 103: val_loss improved from 2.55274 to 2.55090, saving model to dense_transfer_learning_model.h5
4735/4735 [==============================] - 692s 146ms/step - loss: 2.5863 - accuracy: 0.3282 - val_loss: 2.5509 - val_accuracy: 0.3338 - lr: 1.0000e-05
Epoch 104/300
4735/4735 [==============================] - ETA: 0s - loss: 2.5863 - accuracy: 0.3275
Epoch 104: val_loss did not improve from 2.55090
4735/4735 [==============================] - 696s 147ms/step - loss: 2.5863 - accuracy: 0.3275 - val_loss: 2.5551 - val_accuracy: 0.3322 - lr: 1.0000e-05
Epoch 105/300
4735/4735 [=======================

In [ ]:
end_time = time.time()
training_time = end_time - start_time
print(f"Total training time: {training_time:.2f} seconds")

In [ ]:
import matplotlib.pyplot as plt


# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.grid()

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Model names
models = ["VGG16", "VGG19", "GoogleNet", "ResNet", "DenseNet", "MobileNet", 
          "EfficientNetB0", "EfficientNetB7", "Inception-Res", "XceptionNet"]

# Training and Validation accuracies
training_accuracies = [44.00, 39.42, 53.10, 73.29, 61.19, 65.18, 76.73, 82.34, 58.34, 59.63]
validation_accuracies = [42.90, 39.25, 53.58, 65.63, 60.96, 61.02, 70.77, 79.38, 58.60, 58.74]

# Plotting
bar_width = 0.35
index = np.arange(len(models))

fig, ax = plt.subplots(figsize=(10, 6))
bar1 = ax.bar(index, training_accuracies, bar_width, label='Training Accuracy')
bar2 = ax.bar(index + bar_width, validation_accuracies, bar_width, label='Validation Accuracy')

ax.set_xlabel('Models')
ax.set_ylabel('Accuracy (%)')
ax.set_title('Training and Validation Accuracies of Different Models Epochs = 30')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(models, rotation=45, ha='right')
ax.legend()

plt.tight_layout()
plt.show()
